# Draft Testing Notebook

This notebook is used for testing draft implementations.

## 2026-01-15: Structural Constraint Draft Testing

In [ ]:
import sys
sys.path.insert(0, '../src')

import pub_utils as pu
from pub_utils import structural_constraint_draft as sc
import pandas as pd
import numpy as np

## Test 1: Load structural connectomes

In [ ]:
# List available datasets
datasets = sc.get_available_structural_datasets()
print("Chemical synapse datasets:", datasets['chemical_synapse'])
print("Electrical synapse datasets:", datasets['electrical_synapse'])

In [ ]:
# Load chemical synapse connectome
chem = sc.load_structural_connectome('chemical', 'Cook2019')
print(f"Chemical connectome shape: {chem.shape}")
print(f"Total connections: {(chem > 0).sum().sum()}")

In [ ]:
# Load electrical synapse connectome (should be symmetrized)
elec = sc.load_structural_connectome('electrical', 'Cook2019')
print(f"Electrical connectome shape: {elec.shape}")
print(f"Total connections: {(elec > 0).sum().sum()}")

# Verify symmetry
is_symmetric = np.allclose(elec.values, elec.values.T)
print(f"Is symmetric (after symmetrization): {is_symmetric}")

In [ ]:
# Load combined (both)
both = sc.load_structural_connectome('both', 'Cook2019')
print(f"Combined connectome shape: {both.shape}")
print(f"Total connections: {(both > 0).sum().sum()}")

## Test 2: Apply structural constraint to molecular connectome

In [ ]:
# Assemble dopamine molecular connectome
dop_conn = pu.assemble_nt_connectome(
    'dopamine',
    release_markers=['synthesis'],
    release_sources=['literature:Bentley2016'],
    receptor_sources=['reporter:Muralidhara2025']
)
dop_binary = dop_conn['binary']
print(f"Dopamine connectome shape: {dop_binary.shape}")
print(f"Molecular connections: {(dop_binary > 0).sum().sum()}")

In [ ]:
# Apply structural constraint (binary mode)
constrained = sc.apply_structural_constraint(dop_binary, chem, mode='binary')
print(f"Constrained connections: {(constrained > 0).sum().sum()}")

# Compare
comparison = sc.compare_constrained_vs_unconstrained(dop_binary, constrained)
print(f"\nComparison:")
print(f"  Original: {comparison['original_connections']}")
print(f"  Constrained: {comparison['constrained_connections']}")
print(f"  Removed: {comparison['removed_connections']}")
print(f"  Retention rate: {comparison['retention_rate']:.1%}")

In [ ]:
# Use high-level convenience function
result = sc.constrain_molecular_connectome(
    dop_binary,
    structural_dataset='Cook2019',
    synapse_type='both',  # chemical + electrical
    mode='binary'
)

print(f"Constrained (both synapse types): {(result['constrained'] > 0).sum().sum()}")
print(f"\nMetadata: {result['metadata']}")

In [ ]:
# Test weighted mode
result_weighted = sc.constrain_molecular_connectome(
    dop_binary,
    structural_dataset='Cook2019',
    synapse_type='chemical',
    mode='weighted'
)

print(f"Weighted constraint - non-zero entries: {(result_weighted['constrained'] > 0).sum().sum()}")
print(f"Max weighted value: {result_weighted['constrained'].max().max()}")

## Summary

The structural constraint draft provides:
1. `load_structural_connectome()` - Load chemical/electrical/both connectomes
2. `apply_structural_constraint()` - Binary or weighted constraint application
3. `constrain_molecular_connectome()` - High-level convenience function
4. `compare_constrained_vs_unconstrained()` - Compare before/after statistics
5. `get_available_structural_datasets()` - List available datasets